In [5]:
# !pip install nltk
# !pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 11.1/11.1 MB 6.4 MB/s eta 0:00:00
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.14.1-cp310-cp310-win_amd64.whl (44.8 MB)
     ---------------------------------------- 12.9/12.9 MB 2.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import nltk
from nltk.corpus import brown
from sklearn.model_selection import train_test_split

In [18]:
# Download the Brown corpus
nltk.download('brown')

# Load the tagged sentences from the "news" category
tagged_sents = brown.tagged_sents(categories='news')

[nltk_data] Downloading package brown to /Users/tomershav/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [19]:
train_data, test_data = train_test_split(
    tagged_sents, test_size=0.1, shuffle=False
)

# Display dataset sizes
print(f"Number of sentences in training set: {len(train_data)}")
print(f"Number of sentences in testing set: {len(test_data)}")

# Save the training and testing datasets for further usage
with open("train_data.pkl", "wb") as train_file, open("test_data.pkl", "wb") as test_file:
    import pickle
    pickle.dump(train_data, train_file)
    pickle.dump(test_data, test_file)

print("Dataset preparation complete.")

Number of sentences in training set: 4160
Number of sentences in testing set: 463
Dataset preparation complete.


In [20]:
from collections import defaultdict

# Step 1: Compute p(tag|word) using the training set
tag_counts = defaultdict(int)
word_tag_counts = defaultdict(lambda: defaultdict(int))

for sentence in train_data:
    for word, tag in sentence:
        tag_counts[tag] += 1
        word_tag_counts[word][tag] += 1

# Step 2: Determine the most likely tag for each word
most_likely_tag = {}
for word, tags in word_tag_counts.items():
    most_likely_tag[word] = max(tags, key=tags.get)

# Step 3: Define a function to tag a sentence
def tag_sentence(sentence):
    """
    Tags a sentence using the most likely tag baseline.
    """
    tagged_sentence = []
    for word in sentence:
        # Use the most likely tag for known words; default to 'NN' for unknown words
        tag = most_likely_tag.get(word, 'NN')
        tagged_sentence.append((word, tag))
    return tagged_sentence

# Step 4: Evaluate on the test set
known_correct = 0
unknown_correct = 0
known_total = 0
unknown_total = 0

for sentence in test_data:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    # Predict tags for the sentence
    predicted_tags = [tag for word, tag in tag_sentence(words)]

    for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tags):
        if word in most_likely_tag:  # Known word
            known_total += 1
            if true_tag == predicted_tag:
                known_correct += 1
        else:  # Unknown word
            unknown_total += 1
            if true_tag == predicted_tag:
                unknown_correct += 1

# Step 5: Calculate error rates
known_error_rate = 1 - (known_correct / known_total)
unknown_error_rate = 1 - (unknown_correct / unknown_total)
total_error_rate = 1 - ((known_correct + unknown_correct) / (known_total + unknown_total))

# Step 6: Print results
print(f"Known Words Error Rate: {known_error_rate:.4f}")
print(f"Unknown Words Error Rate: {unknown_error_rate:.4f}")
print(f"Total Error Rate: {total_error_rate:.4f}")


Known Words Error Rate: 0.0832
Unknown Words Error Rate: 0.7897
Total Error Rate: 0.1639


In [21]:
"""
Known Words Error Rate: 0.0832
Unknown Words Error Rate: 0.7897
Total Error Rate: 0.1639
"""

'\nKnown Words Error Rate: 0.0832\nUnknown Words Error Rate: 0.7897\nTotal Error Rate: 0.1639\n'

In [31]:
from collections import defaultdict
import math

# Step (c)i: Training phase
# Compute transition and emission probabilities using Maximum Likelihood Estimation (MLE)

# Initialize dictionaries to count occurrences
bigram_counts = defaultdict(lambda: defaultdict(int))  # Counts of tag bigrams
emission_counts = defaultdict(lambda: defaultdict(int))  # Counts of (word, tag)
tag_counts = defaultdict(int)  # Counts of individual tags

# Count tag bigrams and emissions from training data
for sentence in train_data:
    prev_tag = None  # Beginning of a sentence
    for word, tag in sentence:
        tag_counts[tag] += 1
        emission_counts[tag][word] += 1
        if prev_tag is not None:
            bigram_counts[prev_tag][tag] += 1
        prev_tag = tag

# Compute transition probabilities P(tag2|tag1) and emission probabilities P(word|tag)
transition_probs = defaultdict(lambda: defaultdict(float))
emission_probs = defaultdict(lambda: defaultdict(float))

for prev_tag, next_tags in bigram_counts.items():
    total_bigrams = sum(next_tags.values())
    for next_tag, count in next_tags.items():
        transition_probs[prev_tag][next_tag] = count / total_bigrams

for tag, words in emission_counts.items():
    total_emissions = sum(words.values())
    for word, count in words.items():
        emission_probs[tag][word] = count / total_emissions

# Step (c)ii: Viterbi algorithm without recursion
def viterbi_algorithm(sentence, tag_set, transition_probs, emission_probs, unknown_tag='NN'):
    """
    Perform POS tagging using the Viterbi algorithm for a bigram HMM without recursion.
    """
    n = len(sentence)
    viterbi = [{} for _ in range(n)]  # Viterbi table: list of dictionaries
    backpointer = [{} for _ in range(n)]  # Backpointer table: list of dictionaries

    # Initialization for the first word
    for tag in tag_set:
        viterbi[0][tag] = (
            transition_probs.get(None, {}).get(tag, 1e-10)  # Start transition probability
            * emission_probs.get(tag, {}).get(sentence[0], 1e-10)  # Emission probability
        )
        backpointer[0][tag] = None

    # Fill the Viterbi table for the rest of the sentence
    for t in range(1, n):
        for tag in tag_set:
            max_prob = 0
            best_prev_tag = None
            for prev_tag in tag_set:
                prob = (
                    viterbi[t - 1][prev_tag]  # Probability of the previous state
                    * transition_probs.get(prev_tag, {}).get(tag, 1e-10)  # Transition probability
                    * emission_probs.get(tag, {}).get(sentence[t], 1e-10)  # Emission probability
                )
                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = prev_tag
            viterbi[t][tag] = max_prob
            backpointer[t][tag] = best_prev_tag

    # Backtrack to find the best tag sequence
    best_final_tag = max(tag_set, key=lambda tag: viterbi[n - 1][tag])
    best_tags = [best_final_tag]

    for t in range(n - 1, 0, -1):
        best_tags.insert(0, backpointer[t][best_tags[0]])

    return list(zip(sentence, best_tags))

# Step (c)iii: Run the Viterbi algorithm on the test set and evaluate
correct = 0
total = 0
known_correct = 0
known_total = 0
unknown_correct = 0
unknown_total = 0

all_tags = set(tag_counts.keys())

for sentence in test_data:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    # Predict tags using the Viterbi algorithm
    predicted_tags = [tag for word, tag in viterbi_algorithm(words, all_tags, transition_probs, emission_probs)]

    # Evaluate
    for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tags):
        total += 1
        if true_tag == predicted_tag:
            correct += 1

        if word in emission_probs:
            known_total += 1
            if true_tag == predicted_tag:
                known_correct += 1
        else:
            unknown_total += 1
            if true_tag == predicted_tag:
                unknown_correct += 1

In [32]:

# Calculate error rates
known_error_rate = 1 - (known_correct / known_total)
unknown_error_rate = 1 - (unknown_correct / unknown_total)
total_error_rate = 1 - (known_correct + unknown_correct) / (known_total + unknown_total)

# Print results
print(f"Known Words Error Rate: {known_error_rate:.4f}")
print(f"Unknown Words Error Rate: {unknown_error_rate:.4f}")
print(f"Total Error Rate: {total_error_rate:.4f}")


Known Words Error Rate: 0.0130
Unknown Words Error Rate: 0.1482
Total Error Rate: 0.1317


In [ ]:
"""
Known Words Error Rate: 0.0130
Unknown Words Error Rate: 0.1482
Total Error Rate: 0.1317
"""

In [30]:
from collections import defaultdict

# Step (d)i: Training phase with Add-one smoothing for emission probabilities
# Count tag occurrences and word-tag pair occurrences
emission_counts = defaultdict(lambda: defaultdict(int))  # Counts of (word, tag)
tag_counts = defaultdict(int)  # Counts of individual tags
vocabulary = set()  # Vocabulary of words in the training data

for sentence in train_data:
    for word, tag in sentence:
        tag_counts[tag] += 1
        emission_counts[tag][word] += 1
        vocabulary.add(word)

# Compute emission probabilities with Add-one smoothing
emission_probs_smoothed = defaultdict(lambda: defaultdict(float))

for tag, words in emission_counts.items():
    # Total number of words emitted by the tag (with smoothing)
    total_emissions = tag_counts[tag] + len(vocabulary)  # Add-one smoothing

    for word in vocabulary:
        # Add-one smoothed emission probability for known words
        emission_probs_smoothed[tag][word] = (emission_counts[tag][word] + 1) / total_emissions

    # Add-one smoothed probability for unknown words
    emission_probs_smoothed[tag]["<UNK>"] = 1 / total_emissions

# Step (d)ii: Run the modified Viterbi algorithm on the test set
def viterbi_with_smoothing(sentence, tag_set, transition_probs, emission_probs_smoothed, unknown_tag='NN'):
    """
    Perform POS tagging using the Viterbi algorithm with smoothed emission probabilities.
    """
    n = len(sentence)
    viterbi = [{} for _ in range(n)]  # Viterbi table: list of dictionaries
    backpointer = [{} for _ in range(n)]  # Backpointer table: list of dictionaries

    # Initialization for the first word
    for tag in tag_set:
        viterbi[0][tag] = (
            transition_probs.get(None, {}).get(tag, 1e-10)  # Start transition probability
            * emission_probs_smoothed.get(tag, {}).get(sentence[0], emission_probs_smoothed[tag]["<UNK>"])  # Smoothed emission
        )
        backpointer[0][tag] = None

    # Fill the Viterbi table for the rest of the sentence
    for t in range(1, n):
        for tag in tag_set:
            max_prob = 0
            best_prev_tag = None
            for prev_tag in tag_set:
                prob = (
                    viterbi[t - 1][prev_tag]  # Probability of the previous state
                    * transition_probs.get(prev_tag, {}).get(tag, 1e-10)  # Transition probability
                    * emission_probs_smoothed.get(tag, {}).get(sentence[t], emission_probs_smoothed[tag]["<UNK>"])  # Smoothed emission
                )
                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = prev_tag
            viterbi[t][tag] = max_prob
            backpointer[t][tag] = best_prev_tag

    # Backtrack to find the best tag sequence
    best_final_tag = max(tag_set, key=lambda tag: viterbi[n - 1][tag])
    best_tags = [best_final_tag]

    for t in range(n - 1, 0, -1):
        best_tags.insert(0, backpointer[t][best_tags[0]])

    return list(zip(sentence, best_tags))

# Evaluate on the test set
correct = 0
total = 0
known_correct = 0
known_total = 0
unknown_correct = 0
unknown_total = 0

for sentence in test_data:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    # Predict tags using the Viterbi algorithm with smoothed emission probabilities
    predicted_tags = [tag for word, tag in viterbi_with_smoothing(words, all_tags, transition_probs, emission_probs_smoothed)]

    # Evaluate
    for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tags):
        total += 1
        if true_tag == predicted_tag:
            correct += 1

        if word in vocabulary:
            known_total += 1
            if true_tag == predicted_tag:
                known_correct += 1
        else:
            unknown_total += 1
            if true_tag == predicted_tag:
                unknown_correct += 1

# Calculate error rates
known_error_rate = 1 - (known_correct / known_total)
unknown_error_rate = 1 - (unknown_correct / unknown_total)
total_error_rate = 1 - (correct / total)

# Print results
print(f"Known Words Error Rate: {known_error_rate:.4f}")
print(f"Unknown Words Error Rate: {unknown_error_rate:.4f}")
print(f"Total Error Rate: {total_error_rate:.4f}")


Known Words Error Rate: 0.1813
Unknown Words Error Rate: 0.7609
Total Error Rate: 0.2475


In [33]:
"""
Known Words Error Rate: 0.1813
Unknown Words Error Rate: 0.7609
Total Error Rate: 0.2475
"""

'\nKnown Words Error Rate: 0.1813\nUnknown Words Error Rate: 0.7609\nTotal Error Rate: 0.2475\n'

In [42]:
import re
from collections import defaultdict

def get_pseudo_word(word):
    """
    Categorize words into pseudo-words based on patterns.
    """
    if re.match(r'^[A-Z][a-z]*$', word):  # Proper noun
        return "<CAPITALIZED>"
    elif re.match(r'^[0-9]+$', word):  # Numeric
        return "<NUMERIC>"
    elif re.match(r'.*ing$', word):  # Gerund
        return "<GERUND>"
    elif re.match(r'.*ed$', word):  # Past tense
        return "<PAST>"
    elif re.match(r'.*s$', word):  # Plural
        return "<PLURAL>"
    elif re.match(r'[!?.]$', word):  # Punctuation
        return "<PUNCTUATION>"
    else:  # General unknown
        return "<UNKNOWN>"

# Threshold for low-frequency words
low_freq_threshold = 2^30
word_counts = defaultdict(int)

# Count word frequencies
for sentence in train_data:
    for word, tag in sentence:
        word_counts[word] += 1

# Replace low-frequency words and unknown words with pseudo-words
train_data_pseudo = [
    [
        (word if word_counts[word] > low_freq_threshold else get_pseudo_word(word), tag)
        for word, tag in sentence
    ]
    for sentence in train_data
]

# Build the vocabulary of frequent words
vocabulary_pseudo = {word for word in word_counts if word_counts[word] > low_freq_threshold}


In [43]:
# Step 1: Train the model using pseudo-words
emission_counts_pseudo = defaultdict(lambda: defaultdict(int))
tag_counts_pseudo = defaultdict(int)

for sentence in train_data_pseudo:
    for word, tag in sentence:
        tag_counts_pseudo[tag] += 1
        emission_counts_pseudo[tag][word] += 1

# Compute emission probabilities for pseudo-words
emission_probs_pseudo = defaultdict(lambda: defaultdict(float))
for tag, words in emission_counts_pseudo.items():
    total_emissions = sum(words.values())
    for word, count in words.items():
        emission_probs_pseudo[tag][word] = count / total_emissions

# Step 2: Replace unknown words in the test set with pseudo-words
test_data_pseudo = [
    [
        (word if word in vocabulary_pseudo else get_pseudo_word(word), tag)
        for word, tag in sentence
    ]
    for sentence in test_data
]

# Step 3: Run Viterbi with pseudo-words
correct, total = 0, 0
known_correct, known_total = 0, 0
unknown_correct, unknown_total = 0, 0

for sentence in test_data_pseudo:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    predicted_tags = [
        tag for word, tag in viterbi_algorithm(words, all_tags, transition_probs, emission_probs_pseudo)
    ]

    for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tags):
        total += 1
        if true_tag == predicted_tag:
            correct += 1

        if word in vocabulary_pseudo:
            known_total += 1
            if true_tag == predicted_tag:
                known_correct += 1
        else:
            unknown_total += 1
            if true_tag == predicted_tag:
                unknown_correct += 1

# Compute error rates
known_error_rate = 1 - (known_correct / known_total)
unknown_error_rate = 1 - (unknown_correct / unknown_total)
total_error_rate = 1 - (correct / total)

print(f"(e)ii) Known Words Error Rate: {known_error_rate:.4f}")
print(f"(e)ii) Unknown Words Error Rate: {unknown_error_rate:.4f}")
print(f"(e)ii) Total Error Rate: {total_error_rate:.4f}")


(e)ii) Known Words Error Rate: 0.0460
(e)ii) Unknown Words Error Rate: 0.3959
(e)ii) Total Error Rate: 0.1959


In [ ]:
"""
(e)ii) Known Words Error Rate: 0.0472
(e)ii) Unknown Words Error Rate: 0.3762
(e)ii) Total Error Rate: 0.1311
"""

In [ ]:
# Step 1: Apply Add-One Smoothing to Pseudo-Words
emission_probs_smoothed_pseudo = defaultdict(lambda: defaultdict(float))

for tag, words in emission_counts_pseudo.items():
    total_emissions = tag_counts_pseudo[tag] + len(vocabulary_pseudo)  # Add-one smoothing
    for word in vocabulary_pseudo:
        emission_probs_smoothed_pseudo[tag][word] = (emission_counts_pseudo[tag][word] + 1) / total_emissions
    emission_probs_smoothed_pseudo[tag]["<UNK>"] = 1 / total_emissions

# Step 2: Run Viterbi with pseudo-words and smoothed probabilities
correct, total = 0, 0
confusion_matrix = defaultdict(lambda: defaultdict(int))

for sentence in test_data_pseudo:
    words = [word for word, tag in sentence]
    true_tags = [tag for word, tag in sentence]

    predicted_tags = [
        tag for word, tag in viterbi_with_smoothing(words, all_tags, transition_probs, emission_probs_smoothed_pseudo)
    ]

    for word, true_tag, predicted_tag in zip(words, true_tags, predicted_tags):
        total += 1
        confusion_matrix[true_tag][predicted_tag] += 1
        if true_tag == predicted_tag:
            correct += 1

# Compute error rates
total_error_rate = 1 - (correct / total)

print(f"(e)iii) Total Error Rate: {total_error_rate:.4f}")

# Step 3: Build and Investigate the Confusion Matrix
tags_list = sorted(all_tags)
print("\nConfusion Matrix:")
print(f"{'':10s} {' '.join(f'{tag:5s}' for tag in tags_list)}")
for true_tag in tags_list:
    row = [confusion_matrix[true_tag].get(predicted_tag, 0) for predicted_tag in tags_list]
    print(f"{true_tag:10s} {' '.join(f'{val:5d}' for val in row)}")
